In [5]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pandas.io.json import json_normalize

with open('data.json','r') as f:
    data = json.loads(f.read())

df = pd.json_normalize(data, meta='title', record_path=['results'])

df['year'] = df['year'].astype(int)
df['rating'] = pd.to_numeric(df['rating'])
df['metascore'] = pd.to_numeric(df['metascore'])/10
df = df[['title','year', 'rating', 'metascore', 'genre']]

In [6]:
df.head()

,title,year,rating,metascore,genre
0,The Shawshank Redemption,1994,9.3,8.1,Drama
1,The Godfather,1972,9.2,10.0,Crime Drama
2,The Dark Knight,2008,9.1,8.4,Action Crime Drama
3,The Godfather: Part II,1974,9.0,9.0,Crime Drama
4,12 Angry Men,1957,9.0,9.6,Crime Drama


In [7]:
plt.rcParams['figure.figsize'] = [20, 7]

df.plot( x=["metascore", "rating"], y="title", kind="bar")
plt.show()

ValueError: x must be a label or position

In [8]:
from pandas import Series

df_dec = df[['genre', 'year', 'metascore', 'rating', 'title']]

s = df['genre'].str.split(' ').apply(Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'genre'
del df_dec['genre']
df_dec = df_dec.join(s)
#df_dec.plot(x="genre", y="year", kind="bar", rot=90, fontsize=10)

#df_dec.plot(x="genre", y=["metascore", "rating"], kind="bar")

df_dec = df_dec.groupby(["year", "genre"], sort=True)["year"].count()
print(df_dec)
d2 = df_dec.to_frame(name = 'genre count').reset_index()

print(d2)

year  genre    
1931  Comedy       1
      Crime        1
      Drama        1
      Mystery      1
      Romance      1
                  ..
2019  Drama        3
      Thriller     2
2021  Action       1
      Adventure    1
      Fantasy      1
Name: year, Length: 212, dtype: int64
     year      genre  genre count
0    1931     Comedy            1
1    1931      Crime            1
2    1931      Drama            1
3    1931    Mystery            1
4    1931    Romance            1
..    ...        ...          ...
207  2019      Drama            3
208  2019   Thriller            2
209  2021     Action            1
210  2021  Adventure            1
211  2021    Fantasy            1

[212 rows x 3 columns]


In [15]:
from matplotlib import rcParams


sns.catplot(x='year', y='genre count', col='genre', data=d2, kind='bar', height=8.27, aspect=11.7/8.27)
sns.despine()

ValueError: Image size of 174240x792 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 174240x792 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 174240x792 with 20 Axes>

In [ ]:
plt.rcParams['figure.figsize'] = [10, 25]

df.plot(x='title', y=['metascore', 'rating'], kind='barh', align='edge', fontsize=16)

plt.show()